In [13]:
from flask import Flask, render_template, request, jsonify
import json
from sympy import Eq
from latex2sympy2 import latex2sympy
from sympy.printing.mathml import mathml

app = Flask(__name__)

# Example `inverted_articles` dictionary for demonstration
inverted_articles = {
    "Function1": {
        "Category1": ["Article1.html", "Article2.html"],
        "Category2": ["Article3.html"]
    },
    "Function2": {
        "Category3": ["Article4.html"],
        "Category4": ["Article5.html", "Article6.html"]
    }
}

@app.route('/')
def index():
    return render_template('index.html', inverted_articles=json.dumps(inverted_articles))

@app.route('/data')
def get_data():
    return jsonify(inverted_articles)

@app.route('/process_latex', methods=['POST'])
def process_latex():
    try:
        data = request.get_json()
        latex_equation = data.get('latex', '')

        # Convert LaTeX to SymPy
        sympy_expr = latex2sympy(latex_equation)

        if "=" in latex_equation:
            # Split the left-hand side and right-hand side, and create a SymPy equation
            lhs, rhs = latex_equation.split("=")
            sympy_expr = Eq(latex2sympy(lhs.strip()),latex2sympy(rhs.strip()))

        # Convert SymPy to ContentMathML
        content_mathml = mathml(sympy_expr, printer='content')
        print(content_mathml)
        return jsonify({'contentMathML': content_mathml})
    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
